In [1]:
from autogluon import TabularPrediction as task
import time

/home/ubuntu/anaconda3/envs/autogluon/lib/python3.7/site-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [2]:
dataset = 'dataset'
!kaggle competitions download -p {dataset} -q otto-group-product-classification-challenge
!unzip -d {dataset} {dataset}/otto-group-product-classification-challenge.zip
!rm {dataset}/otto-group-product-classification-challenge.zip

Archive:  dataset/otto-group-product-classification-challenge.zip
replace dataset/sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
autogluon.

In [3]:
# Training time:
train_data = task.Dataset(file_path=f'{dataset}/train.csv').drop('id', axis=1)
train_data.head()

Loaded data from: dataset/train.csv | Columns = 95 / 95 | Rows = 61878 -> 61878


,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [4]:
label_column = 'target' # specifies which column do we want to predict
savedir = 'otto_models/' # where to save trained models

In [5]:
predictor = task.fit(train_data=train_data, 
                     label=label_column, 
                     output_directory=savedir, 
                     eval_metric='log_loss', 
                     auto_stack=True,
                     verbosity=2,
                     visualizer='tensorboard')

Beginning AutoGluon training ...
AutoGluon will save models to otto_models/
Train Data Rows:    61878
Train Data Columns: 94
Preprocessing data ...
Here are the first 10 unique label values in your data:  ['Class_1' 'Class_2' 'Class_3' 'Class_4' 'Class_5' 'Class_6' 'Class_7'
 'Class_8' 'Class_9']
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == object)
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Feature Generator processed 61878 data points with 93 features
Original Features:
	int features: 93
Generated Features:
	int features: 0
All Features:
	int features: 93
	Data preprocessing and feature engineering runtime = 0.36s ...
AutoGluon will gauge predictive performance using evaluation metric: log_loss
This metric expects predicted probabilities rather than predicted class labels, so you'll need to use predict_proba() instead of predict

In [6]:
results = predictor.fit_summary() # display detailed summary of fit() process

*** Summary of fit() ***
Number of models trained: 22
Types of models trained: 
{'WeightedEnsembleModel', 'StackerEnsembleModel'}
Validation performance of individual models: {'RandomForestClassifierGini_STACKER_l0': -0.5691089791208548, 'RandomForestClassifierEntr_STACKER_l0': -0.5826890390024927, 'ExtraTreesClassifierGini_STACKER_l0': -0.5855208676626903, 'ExtraTreesClassifierEntr_STACKER_l0': -0.6025590380356861, 'KNeighborsClassifierUnif_STACKER_l0': -2.410849865487888, 'KNeighborsClassifierDist_STACKER_l0': -2.4031890499617976, 'LightGBMClassifier_STACKER_l0': -0.466066913889447, 'CatboostClassifier_STACKER_l0': -0.47099706491836313, 'NeuralNetClassifier_STACKER_l0': -0.5984127911895897, 'LightGBMClassifierCustom_STACKER_l0': -0.4595365541478719, 'weighted_ensemble_k0_l1': -0.4420447748019833, 'RandomForestClassifierGini_STACKER_l1': -0.4720540352120622, 'RandomForestClassifierEntr_STACKER_l1': -0.4627935758444223, 'ExtraTreesClassifierGini_STACKER_l1': -0.4778061258079188, 'Extra

In [2]:
# predictor = task.load('otto_models')

In [4]:
dataset = 'dataset'
test_data_full = task.Dataset(file_path=f'{dataset}/test.csv')
test_data = test_data_full.drop('id', axis=1)

Loaded data from: dataset/test.csv | Columns = 94 / 94 | Rows = 144368 -> 144368


In [6]:
%%time
pred_probablities = predictor.predict_proba(test_data, as_pandas=True)

/home/ubuntu/anaconda3/envs/autogluon/lib/python3.7/imp.py:342: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  return _load(spec)


CPU times: user 13d 7h 4min 2s, sys: 14min 38s, total: 13d 7h 18min 40s
Wall time: 3h 36min


In [7]:
pred_probablities.insert(loc=0, column='id', value=test_data_full['id'])

In [10]:
submission_name = f'submission-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}.csv'
pred_probablities.to_csv(submission_name,index=False)
!kaggle competitions submit otto-group-product-classification-challenge -f {submission_name} -m "autogluon {submission_name}"

100%|██████████████████████████████████████| 27.4M/27.4M [00:02<00:00, 10.4MB/s]
Successfully submitted to Otto Group Product Classification Challenge